In [1]:
# A Convolutional Network implementation example using TensorFlow library.
# This example is using the MNIST database of handwritten digits
# (http://yann.lecun.com/exdb/mnist/)

# Author: Aymeric Damien
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [13]:
# Import MINST data
import input_data_2
mnist = input_data_2.read_data_sets(num_training=100, num_val=10, num_test=10, one_hot=True)

Shapes:  (397, 3, 150, 150) (397,)
(100, 3, 150, 150) (10, 3, 150, 150) (10, 3, 150, 150)
(100, 150, 150, 3) (10, 150, 150, 3) (10, 150, 150, 3)


In [3]:
import tensorflow as tf

In [4]:
# Parameters
learning_rate = 0.001
training_iters = 1000
batch_size = 20
display_step = 1

In [5]:
# Network Parameters
n_input = 150*150*3 # MNIST data input (img shape: 28*28)
n_classes = 4 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

In [6]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [7]:
# Create model
def conv2d(img, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w, strides=[1, 1, 1, 1], 
                                                  padding='SAME'),b))

def max_pool(img, k):
    return tf.nn.max_pool(img, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(_X, _weights, _biases, _dropout):
    # Reshape input picture
    _X = tf.reshape(_X, shape=[-1, 150, 150, 3])

    # Convolution Layer
    conv1 = conv2d(_X, _weights['wc1'], _biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = max_pool(conv1, k=2)
    # Apply Dropout
    conv1 = tf.nn.dropout(conv1, _dropout)

    # Convolution Layer
    conv2 = conv2d(conv1, _weights['wc2'], _biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = max_pool(conv2, k=2)
    # Apply Dropout
    conv2 = tf.nn.dropout(conv2, _dropout)

    # Fully connected layer
    # Reshape conv2 output to fit dense layer input
    dense1 = tf.reshape(conv2, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    # Relu activation
    dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, _weights['wd1']), _biases['bd1']))
    # Apply Dropout
    dense1 = tf.nn.dropout(dense1, _dropout) # Apply Dropout

    # Output, class prediction
    out = tf.add(tf.matmul(dense1, _weights['out']), _biases['out'])
    return out

In [8]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 3, 32])), 
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])), 
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([38*38*64, 1024])), 
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes])) 
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [9]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)

In [10]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [11]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [12]:
# Initializing the variables
init = tf.initialize_all_variables()

In [13]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"
    # Calculate accuracy for 256 mnist test images
    print "Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], 
                                                             y: mnist.test.labels[:256], 
                                                             keep_prob: 1.})

Iter 20, Minibatch Loss= 416680.406250, Training Accuracy= 0.30000
Iter 40, Minibatch Loss= 351253.312500, Training Accuracy= 0.15000
Iter 60, Minibatch Loss= 135322.500000, Training Accuracy= 0.45000
Iter 80, Minibatch Loss= 277173.343750, Training Accuracy= 0.35000
Iter 100, Minibatch Loss= 318175.156250, Training Accuracy= 0.30000
Iter 120, Minibatch Loss= 498504.562500, Training Accuracy= 0.30000
Iter 140, Minibatch Loss= 328963.687500, Training Accuracy= 0.50000
Iter 160, Minibatch Loss= 358024.843750, Training Accuracy= 0.45000
Iter 180, Minibatch Loss= 287977.281250, Training Accuracy= 0.40000
Iter 200, Minibatch Loss= 224046.656250, Training Accuracy= 0.35000
Iter 220, Minibatch Loss= 90477.140625, Training Accuracy= 0.60000
Iter 240, Minibatch Loss= 220186.296875, Training Accuracy= 0.45000
Iter 260, Minibatch Loss= 175401.093750, Training Accuracy= 0.45000
Iter 280, Minibatch Loss= 268298.906250, Training Accuracy= 0.45000
Iter 300, Minibatch Loss= 430568.500000, Training Acc